In [ ]:
!gdown 1S8eoifR55jkj0hbDSvLIi8ppBBMaj9fK

Downloading...
From: https://drive.google.com/uc?id=1S8eoifR55jkj0hbDSvLIi8ppBBMaj9fK
To: /content/archive.zip
100% 213M/213M [00:02<00:00, 72.9MB/s]


In [ ]:
!unzip /content/archive.zip >> unarchiving.log

In [ ]:
#Converting the labels from the given xml to the yolov5 format with normalization


import os
import xml.etree.ElementTree as ET
from glob import glob
from tqdm import tqdm


def convert_xml_to_txt(xml_path, txt_path):
    myroot = ET.parse(xml_path).getroot()
    file_name = myroot.find('filename').text
    
    img_width = int(myroot.find('size').find('width').text)
    img_height = int(myroot.find('size').find('height').text)

    objects = myroot.findall('object')

    txt_path = os.path.join(txt_path, file_name.split('.')[0] + '.txt')

    with open(txt_path, 'w') as f:
        
        for bbox in objects:
            
            bndbox = bbox.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height

            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            f.write(f'0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n')


if not os.path.exists('/content/labels'):
    os.makedirs('/content/labels')

output_path = '/content/labels/'
annotations = glob('/content/annotations/*.xml')

for annotation in tqdm(annotations):
    convert_xml_to_txt(annotation, output_path)


100%|██████████| 433/433 [00:00<00:00, 7887.08it/s]


In [ ]:
yaml_data = """# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../images/
val: ../images/

# number of classes
nc: 1

# class names
names: ['license']
"""
f = open("/content/data.yaml", "w")
print(yaml_data, file=f)
f.close()

In [ ]:
!cat /content/data.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../images/
val: ../images/

# number of classes
nc: 1

# class names
names: ['license']



In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 11888, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 11888 (delta 40), reused 44 (delta 13), pack-reused 11800
Receiving objects: 100% (11888/11888), 12.52 MiB | 30.31 MiB/s, done.
Resolving deltas: 100% (8149/8149), done.


In [ ]:
%cd yolov5
!pip3 install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.6 MB/s 


In [ ]:
!python3 train.py --img 640 --batch 16 --epochs 30 --data /content/data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-34-ge0700cc Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v

In [ ]:
!python3 detect.py --source /content/images/Cars0.png --weights /content/yolov5/runs/train/exp4/weights/best.pt

detect: weights=['/content/yolov5/runs/train/exp4/weights/best.pt'], source=/content/images/Cars0.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.2-34-ge0700cc Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/images/Cars0.png: 352x640 1 license, 12.6ms
Speed: 0.4ms pre-process, 12.6ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp3


usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--data DATA] [--imgsz IMGSZ [IMGSZ ...]]
                 [--conf-thres CONF_THRES] [--iou-thres IOU_THRES]
                 [--max-det MAX_DET] [--device DEVICE] [--view-img]
                 [--save-txt] [--save-conf] [--save-crop] [--nosave]
                 [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--visualize] [--update] [--project PROJECT]
                 [--name NAME] [--exist-ok] [--line-thickness LINE_THICKNESS]
                 [--hide-labels] [--hide-conf] [--half] [--dnn]

optional arguments:
  -h, --help            show this help message and exit
  --weights WEIGHTS [WEIGHTS ...]
                        model path(s)
  --source SOURCE       file/dir/URL/glob, 0 for webcam
  --data DATA           (optional) dataset.yaml path
  --imgsz IMGSZ [IMGSZ ...], --img IMGSZ [IMGSZ ...], --img-size IMGSZ [IMGSZ ...]
                        inference size